In [1]:
import os
import pandas as pd
import gc
import gc
import glob
import mimetypes
from collections import Counter
from urlextract import URLExtract
root_dir = os.path.abspath(os.curdir)
dir_coms = os.path.dirname(root_dir)+"/ExtractedComments/"
#Create directory to save experiment results
comlinks_dir = os.path.dirname(root_dir)+"/CommentLinks/"
os.makedirs(comlinks_dir, exist_ok=True)

In [2]:
#Extract links
extractor = URLExtract()
def Find(string):
    try:
        url = extractor.find_urls(str(string))  
    except:
        url = []
    return url

fields = ["ID","Author","LinkID","Subreddit","Text","Time"]
for year in range(2006,2008):
    print(year)
    file = dir_coms+str(year)+'.csv'
    df = pd.read_csv(file,header=None,names=fields)
    df['Links'] = df.Text.apply(lambda x: Find(x))
    df = df[df.Links.astype(bool)]
    df = df[["ID","Author","Subreddit","Time","Links"]]
    output = comlinks_dir+str(year)+'.csv'
    df.to_csv(output,index=False)

2006
2007


In [5]:
#Test 1, count images
counter = 0
imCounter = 0
for year in range(2006,2008):
    print(year)
    file = comlinks_dir+str(year)+'.csv'
    with open(file,'r', encoding='utf-8') as fin:
        first = fin.readline()
        for line in fin:
            counter+=1
            data = line.strip().split(',')
            try:
                links = [x for x in data[4][1:-1].replace("'", '').replace('"','').split(",")]
                #links = ','.join(lin)
                # print(links)
                for item in links:
                    medtype = mimetypes.MimeTypes().guess_type(item)[0]
                    if medtype != None and 'image/' in medtype:
                        #print(links)
                        imCounter+=1
            except IndexError:
                links = []
                
    print("Total links:",counter)
    print("Total images:",imCounter)
    counter=0
    imCounter=0

#Test2 count reddit links vs other links (internal & external)
counter = 0
inCounter = 0
outCounter=0
for year in range(2006,2008):
    print(year)
    file = comlinks_dir+str(year)+'.csv'
    with open(file,'r', encoding='utf-8') as fin:
        for line in fin:
            counter+=1
            data = line.strip().split(',')
            try:
                links = [x for x in data[4][1:-1].replace("'", '').replace('"','').split(",")]
                #links = ','.join(lin)
                # print(links)
                for item in links:
                    #print(item)
                    if 'www.reddit.com/r/' in item or 'redd.it' in item or 'www.reddit.com/user/' in item:
                        inCounter+=1
                    elif 'reddit.com/s/' in links:
                        outCounter+=1
                    else:
                        outCounter+=1
            except IndexError:
                links = []
    print("Total available links:",counter)
    pre1 = (inCounter/counter)*100
    print("Total internal links:",inCounter,'('+str(format(pre1,'.2f'))+'%'+')')
    pre2 = (outCounter/counter)*100
    print("Total external links:",outCounter,'('+str(format(pre2,'.2f'))+'%'+')')
    counter=0
    inCounter=0
    outCounter=0

#Test3 find data types from the links in comments
counter = 0
knownTypes = 0
allMedia=[]
for year in range(2006,2008):
    print(year)
    media=[]
    file = comlinks_dir+str(year)+'.csv'
    with open(file,'r', encoding='utf-8') as fin:
        first = fin.readline()
        for line in fin:
            counter+=1
            data = line.strip().split(',')
            try:
                links = [x for x in data[4][1:-1].replace("'", '').replace('"','').split(",")]
                #links = ','.join(lin)
                # print(links)
                for item in links:
                    medtype = mimetypes.MimeTypes().guess_type(item)[0]
                    if medtype != None:
                        media.append(medtype)
                        allMedia.append(medtype)
                        knownTypes+=1
            except IndexError:
                links = []
            
            
    print("Total links:",counter)
    print("Total known media types:",knownTypes)
    print(dict(Counter(media)))
    counter=0
    knownTypes=0
print("Full collection:")
print(Counter(allMedia))

2006
Total links: 19052
Total images: 556
2007
Total links: 74055
Total images: 3420
2006
Total available links: 19053
Total internal links: 1 (0.01%)
Total external links: 19052 (99.99%)
2007
Total available links: 74056
Total internal links: 2 (0.00%)
Total external links: 74054 (100.00%)
2006
Total links: 19052
Total known media types: 4679
{'text/html': 3724, 'application/pdf': 224, 'image/jpeg': 416, 'text/x-python': 35, 'text/plain': 32, 'image/gif': 85, 'video/quicktime': 20, 'image/png': 52, 'application/x-tar': 2, 'application/msword': 4, 'application/x-shockwave-flash': 3, 'video/x-msvideo': 3, 'application/postscript': 6, 'video/mpeg': 6, 'video/mp4': 1, 'application/x-sh': 2, 'audio/mpeg': 19, 'application/javascript': 6, 'text/xml': 8, 'application/vnd.apple.mpegurl': 4, 'application/octet-stream': 5, 'audio/basic': 3, 'image/svg+xml': 3, 'audio/x-wav': 5, 'application/zip': 5, 'application/vnd.ms-excel': 3, 'application/x-pn-realaudio': 1, 'application/xml': 1, 'applicati